In [20]:
!pip install --upgrade transformers
!pip install --upgrade tf_keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 40.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.0 MB/s eta 0:00:00
     ━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/590.6 MB 81.8 MB/s eta 0:00:07
ERROR: Operation cancelled by user


In [1]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
!pip install transformers datasets

In [ ]:
!wget "https://gitlab.com/accredian/insaid-data/-/raw/main/Term%2012%20Natural%20Language%20Processing/Project%203_%20Classification%20-%20Subject%20Text%20Classification/data.zip"

--2024-07-15 09:53:29--  https://gitlab.com/accredian/insaid-data/-/raw/main/Term%2012%20Natural%20Language%20Processing/Project%203_%20Classification%20-%20Subject%20Text%20Classification/data.zip
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1515682 (1.4M) [application/octet-stream]
Saving to: ‘data.zip.2’

data.zip.2          100%[===================>]   1.45M  --.-KB/s    in 0.01s   

2024-07-15 09:53:29 (134 MB/s) - ‘data.zip.2’ saved [1515682/1515682]



In [2]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
!unzip "data.zip"

Archive:  data.zip
replace data/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/sample_submission.csv  
replace data/subject_topic_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/subject_topic_test.csv  
replace data/subject_topic_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/subject_topic_train.csv  


In [3]:
train_data = pd.read_csv("/content/data/subject_topic_train.csv")
test_data = pd.read_csv("/content/data/subject_topic_test.csv")

In [4]:
train_data

,Id,text,label
0,175,244 Accountancy : Company Accounts and Analy...,accounts
1,631,188 BIOLOGY The transpiration driven asc...,biology
2,1084,MECHANICAL PROPERTIES OF FLUIDS 10.6 10.7 ...,physics
3,481,282 BIOLOGY 18.2 LYMPH (TISSUE FLUID) A...,biology
4,1565,4 4 PHYSICS calculation. Let us take At ...,physics
...,...,...,...
1423,1130,WORK. ENERGY AND POWER 135 6.4 The potential...,physics
1424,1294,LAWS OF MOTION l03 Answer As the string is ...,physics
1425,860,188 wmv BIOLOGY demand of agricultural ...,biology
1426,1459,Electrostatic Potential and Capacitance 2...,physics


In [5]:
train_data['label'].value_counts()

label
physics      620
biology      501
accounts     234
geography     73
Name: count, dtype: int64

In [6]:
category_to_id={"physics":0,"biology":1,"accounts":2,"geography":3}
id_to_category={0:"physics",1:"biology",2:"accounts",3:"geography"}

train_data["label"]=train_data["label"].map(category_to_id)

In [7]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(train_data, test_size=0.2, random_state=42)


In [8]:
train_texts = train_df['text'].tolist()
train_labels = train_df['label'].tolist()
val_texts = val_df['text'].tolist()
val_labels = val_df['label'].tolist()

In [9]:
print(len(set(train_labels)))

4


In [12]:
from transformers import BertTokenizer
import tensorflow as tf
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [17]:
# Convert to TensorFlow datasets

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(len(train_texts)).batch(8)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(8)

In [21]:
from transformers import TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

# Load the BERT model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(train_labels)))

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=3e-5),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=[SparseCategoricalAccuracy('accuracy')]
)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:

# Train the model
history = model.fit(train_dataset, epochs=2, validation_data=val_dataset)

Epoch 1/2
143/143 [==============================] - 1933s 13s/step - loss: 0.3397 - accuracy: 0.9159 - val_loss: 0.0896 - val_accuracy: 0.9860
Epoch 2/2
143/143 [==============================] - 1815s 13s/step - loss: 0.0584 - accuracy: 0.9860 - val_loss: 0.0394 - val_accuracy: 0.9860


In [26]:
model.save_pretrained("my_fine_tuned_model")


In [27]:
model = TFBertForSequenceClassification.from_pretrained('/content/my_fine_tuned_model')


Some layers from the model checkpoint at /content/my_fine_tuned_model were not used when initializing TFBertForSequenceClassification: ['dropout_189']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/my_fine_tuned_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [29]:
test_text = test_data['text'].tolist()


In [30]:
test_encodings = tokenizer(test_text, truncation=True, padding=True, max_length=128)


In [31]:
test_dataset = tf.data.Dataset.from_tensor_slices(dict(test_encodings)).batch(8)


In [32]:
predictions = model.predict(test_dataset).logits


45/45 [==============================] - 178s 4s/step


In [33]:
y = tf.argmax(predictions, axis=1)


In [34]:
y

<tf.Tensor: shape=(358,), dtype=int64, numpy=
array([1, 0, 0, 1, 3, 0, 1, 1, 0, 0, 3, 3, 2, 0, 1, 3, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 1, 2, 2, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 2, 3,
       0, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 0, 0, 1, 2, 0, 0, 0, 2, 0, 1, 1,
       1, 1, 3, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 2, 1, 3, 0, 1, 1, 2, 1, 1,
       0, 0, 0, 3, 1, 0, 0, 1, 0, 2, 3, 1, 0, 3, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       2, 0, 2, 0, 1, 2, 0, 0, 1, 2, 0, 1, 0, 1, 1, 0, 2, 2, 1, 2, 2, 1,
       1, 2, 1, 0, 0, 1, 0, 1, 0, 3, 2, 0, 0, 1, 3, 1, 1, 2, 2, 0, 3, 0,
       2, 0, 1, 1, 3, 0, 0, 2, 1, 1, 0, 1, 1, 1, 2, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 2, 0, 1, 1, 2, 1, 1, 2, 1, 0, 0, 2, 2, 1, 1, 1, 2, 0, 0, 0,
       2, 0, 0, 1, 0, 1, 1, 0, 0, 0, 2, 3, 0, 0, 0, 3, 1, 0, 0, 2, 1, 1,
       3, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 2, 1, 1, 1, 0, 1, 3, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 2, 1, 2, 0, 1, 0, 0, 1, 1,
     

In [35]:
df = pd.DataFrame(y, columns = ["label"])


In [36]:
df

,label
0,1
1,0
2,0
3,1
4,3
...,...
353,1
354,1
355,1
356,1


In [38]:
submission = pd.DataFrame({'Id':test_data['Id'],'label':df['label']})
submission.to_csv('submission.csv', index=False)
